In [1]:
'''
Analysis on how the ownership of a token holder change
'''

'\nAnalysis on how the ownership of a token holder change\n'

In [ ]:
from bs4 import BeautifulSoup
import urllib3
import re
import pandas
import numpy
from time import sleep

In [ ]:
#the base url for etherscan
baseUrl='https://etherscan.io/'
#the connection pool
pool=urllib3.PoolManager()
def html_convert_top100(tokenid, classname):
    '''
    Get the top 100 owners of the coin
    '''
    global baseUrl
    global pool
    funUrl_chart='token/tokenholderchart/'
    r=pool.request('GET',baseUrl+funUrl_chart+tokenid)
    html=r.data
    soup = BeautifulSoup(html)
    table = soup.find("table", attrs=classname)
    headings = [th.get_text() for th in table.find("tr").find_all("th")]
    for idx in range(len(headings)):
        headings[idx]=str(headings[idx])
    datasets = []
    for row in table.find_all("tr")[1:]:
        dataset = (td.get_text() for td in row.find_all("td"))
        ls=list(dataset)
        datasets.append(ls)

    '''
    Clean the data
    '''
    for idx in range(len(datasets)):
        for idxx in range(len(datasets[idx])):
            tmp=re.sub(r'\([^)]*\)', '', datasets[idx][idxx])
            tmp=tmp.strip()
            tmp=str(tmp)
            datasets[idx][idxx]=tmp

    '''
    Create pandas dataframe and convert it to float
    '''
    df=pandas.DataFrame(datasets, columns=headings)
    df['Quantity (Token)']=df['Quantity (Token)'].str.replace(',','')
    df['Quantity (Token)']=df['Quantity (Token)'].astype(numpy.float64)
    df['Percentage']=df['Percentage'].str.strip('%')
    df['Percentage']=df['Percentage'].astype(numpy.float64)/100
    return df

def owners_tr(ownerid, tokenname, classname):
    global baseUrl
    global pool
    transUrl='tokentxns?a='
    nextlinks=[]
    nextlinks.append(transUrl+ownerid)
    while len(nextlinks)>0:
        link=nextlinks.pop()
        r=pool.request('GET',baseUrl+link)
        html=r.data
        soup = BeautifulSoup(html)
        '''
        Get the next link
        '''
        a_tag=soup.find_all('a', id="ContentPlaceHolder1_HyperLinkNext",href=True)
        next_link=a_tag[0]['href']
        nextlinks.append(next_link)
        table = soup.find("table", attrs=classname)
        headings = [th.get_text() for th in table.find("tr").find_all("th")]
        for idx in range(len(headings)):
            headings[idx]=str(headings[idx])
            if headings[idx]=='':
                headings[idx]='direction'
        datasets = []
        for row in table.find_all("tr")[1:]:
            dataset = (td.get_text() for td in row.find_all("td"))
            ls=list(dataset)
            datasets.append(ls)
        '''
        Clean the data
        '''
        for idx in range(len(datasets)):
            for idxx in range(len(datasets[idx])):
                tmp=re.sub(r'\([^)]*\)', '', datasets[idx][idxx])
                tmp=tmp.strip()
                tmp=str(tmp)
                datasets[idx][idxx]=tmp
        '''
        Create pandas dataframe and convert it to float
        '''
        df=pandas.DataFrame(datasets, columns=headings)
        df['Token']=df['Token'].str.upper()
        df=df.loc[df['Token'] == tokenname]
        df['Value']=df['Value'].str.replace(',','')
        df['Value']=df['Value'].astype(numpy.float64)
        '''
        trans_dic={
            tx:{
                Value:'', 
                Block:''
            }
        }
        '''
        trans_dic={}
        for index, row in df.iterrows():
            if row['Value']!=0:
                if row['direction'] == 'OUT':
                    val=-row['Value']
                else:
                    val=row['Value']
                trans_dic[row['TxHash']]={}
                trans_dic[row['TxHash']]['Value']=val
        
        txurl='tx/'
        for tx in trans_dic:
            block=''
            req=pool.request('GET',baseUrl+txurl+tx)
            html_tx=req.data
            tx_soup=BeautifulSoup(html_tx)
            tx_a_tag=tx_soup.find_all('a',href=True)
            for tag in tx_a_tag:
                if '/block/' in tag['href']:
                    block=str(tag.getText())
                    trans_dic[tx]['Block']=block
        print trans_dic
        #set a lag here so the server won't kick us out
        sleep(0.01)
        

def ICO_TOKEN(tokenid, tokenname):

    df=html_convert_top100(tokenid, "table table-hover ")
    '''
    Construct the ownership table
    '''
    owners={}
    for index, row in df.iterrows():
        owners[row['Address']]=row['Quantity (Token)']
    '''
    Construct the ownership transaction table
    '''
    owners_tr("0xd0a6e6c54dbc68db5db3a091b171a77407ff7ccf",tokenname, 'table table-hover ')
        
ICO_TOKEN('0x86fa049857e0209aa7d9e616f7eb3b3b78ecfdb0','EOS')

/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
/usr/local/lib/python2.7/site-packa

{'0x3c9df1b6e8dd4459c4161dce76651b094aa143a8e40704e2389158a22e5818be': {'Block': u'4067917', 'Value': -1.2032906417593603}, '0x1f4f300a19a021aa06017f2d0236fbf96db337bb1bae228015443226d80c11f7': {'Block': u'4067930', 'Value': -117.32602289817083}}


/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is be

{'0x3cb141efc1843b7a68947d9ef34af46bc27da95012c6ebca0caeeead69975485': {'Block': u'4067903', 'Value': -35.91755767393391}, '0x094103c979ec36751b38b59ad78eed03a5e61eca5d85583e104d540e3aa68851': {'Block': u'4067903', 'Value': -862.0213841744139}, '0x9de4a7e7b3d674e8bbffcf1c56bd9741591dd6b0617dd64257410f707830474c': {'Block': u'4067903', 'Value': -718.3511534786783}, '0x38a5d462ebb0c9ff57458737fcec70639d27cb1b8e8d48bd1712330c9929a1a1': {'Block': u'4067907', 'Value': -1149.3618455658852}}


/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'0x78d038e7ea882e9f9012322c6644c1037f36d5a991b362a6862f05dfd9120527': {'Block': u'4067887', 'Value': -223.09914951056763}, '0x50ff7bd0c596a0b3423ec1567e56d2d024b84f43d2a29cd8bef3e41d58c70fa8': {'Block': u'4067901', 'Value': -22.291944350652457}}


/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'0xd3b873e11944419e10a4d23d3c1825cde8221999ca79dc440ca6f3aa40ce6c9a': {'Block': u'4067878', 'Value': -28.73404613914713}, '0x8b7e9a9fb9fa1229d791cb60b89ee0a86f3e3d2cb83a4459cd1fe3d7ca39aa45': {'Block': u'4067880', 'Value': -212.63194142968877}}


/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
